In [1]:
%run mlp_model.ipynb

In [2]:
class AdamModel(MlpModel):
    def __init__(self, name, dataset, hconfigs):
        self.use_adam = False     #먼저 디폴트 값은 False로 설정 필요할 경우만 True로 호출
        super(AdamModel, self).__init__(name, dataset, hconfigs)   #기반 클래스의 객체 초기화 메서드를 호출해 나머지 처리를 맡깁니다.

In [3]:
#신경망 역전파 메서드 재정의
def adam_backprop_layer(self, G_y, hconfig, pm, aux):
    x, y = aux
    
    if hconfig is not None: G_y = relu_derv(y) * G_y
    
    g_y_weight = x.transpose()
    g_y_input = pm['w'].transpose()
    
    G_weight = np.matmul(g_y_weight, G_y)
    G_bias = np.sum(G_y, axis=0)
    G_input = np.matmul(G_y, g_y_input)
    
    self.update_param(pm, 'w',  G_weight) #기존 모델에서 바뀐부분이 이 두줄인데 가중치 파라미터와 바이어스 파라미터를 수정하는 부분이다.
    self.update_param(pm, 'b',  G_bias)   #파라미터값을 직접 고치던 것을 update_param()메서드를 호출해 처리하도록 수정

    return G_input

AdamModel.backprop_layer = adam_backprop_layer

In [4]:
#파라미터 수정 메서드 정의
#이 메서드는 파라미터 수정 과정에서 아담 알고리즘의 선택적 적용을 지원한다.
def adam_update_param(self, pm, key, delta):    #처음 호출될  때 매개변수 delta로 전달된 값은 파라미터의 손실 기울기다.
    if self.use_adam:
        delta = self.eval_adam_delta(pm, key, delta)  #self.use_adam의 플래그 값이 참인 경우 아담 알고리즘을 적용해야 하므로
                                                      #eval_adam_delta() 메서드를 호출하여 delta값에 아담 알고리즘에 따른 수정을 가한다.
    
    pm[key] -= self.learning_rate * delta   #이렇게 결정된 delta값과 self.learning_rate에 저장된 학습률 정보를 이용해 파라미터값을 수정
        
AdamModel.update_param = adam_update_param

In [5]:
#두 가지 파라미터 수정 메서드 정의
#아담 알고리즘에 따라 손실 기울깃값을 보정할 eval_adam_delta() 메서드를 정의한다.

#아담 알고리즘에서는 모멘텀과 2차 모멘텀값을 이동평균 방식으로 유지한다.
def adam_eval_adam_delta(self, pm, key, delta):
    ro_1 = 0.9             #이동평균계산에서 기존 값의 비율을 ro_1, ro_2라는 이름의 하이퍼파라미터로 이용
    ro_2 = 0.999
    epsilon = 1.0e-8       #분모의 임계값 epsilon도 하이퍼파라미터로 이용해 분모가 0이 되는 바람에 생기는 나눗셈 오류를 방지한다.
    
    #모멘텀, 2차 모멘텀값, 처리 횟수를 pm 구조에 저장한다.
    skey, tkey, step = 's' + key, 't' + key, 'n' + key
    if skey not in pm: #파라미터 생성과정에서 모멘텀과 2차 모멘텀 정보를 저장할 버퍼 공간을 마련하지 않았기때문에 두 버퍼공간을 생성해 pm구조에 저장한다.
        pm[skey] = np.zeros(pm[key].shape)
        pm[tkey] = np.zeros(pm[key].shape)
        pm[step] = 0    #미니배치 처리 횟수를 나타내느 step 정보를 0으로 초기화한다.
    
    #이동평균, 즉 기존 값과 새 값의 가중평균을 계산하여 모멘텀값과 2차 모멘텀값을 구한다.
    #다음 단계에 다시 이용할 수 있게 pm[skey], pm[tkey]에 저장한다.
    s = pm[skey] = ro_1 * pm[skey] + (1 - ro_1) * delta
    t = pm[tkey] = ro_2 * pm[tkey] + (1 - ro_2) * (delta * delta)
    
    #다음 코드에서 고쳐가며 이용할 수 있게 임시 변수 s, t에도 저장한다.
    pm[step] += 1
    s = s / (1 - np.power(ro_1, pm[step]))           #비율값에 거듭제곱과 나눗셈 연산을 이용하는 처리를 두어 
    t = t / (1 - np.power(ro_2, pm[step]))           #step값이 작을 때에는 모멘텀값과 2차 모멘텀값을 증폭시켰다.
    
    return s / (np.sqrt(t)+epsilon)                 #2차 모멘텀값의 제곱근으로 1차 모멘텀값을 나눈 비율을 반환해 파라미터값 수정에 이용

AdamModel.eval_adam_delta = adam_eval_adam_delta